Парсер описаний видео и комментариев с facebook.<br>Позволяет собрать данные с определенной страницы (page_id) от заданной даты до текущего дня.<br>Работает через Facebook Graph API

In [1]:
from datetime import datetime
import pandas as pd

import urllib3
import facebook
import requests

In [19]:
# TODO: add start_date
def get_posts_from_page(page_id, token, stop_date):
    stop_date = datetime.strptime(stop_date, '%d.%m.%Y')
    request = 'https://graph.facebook.com/'+page_id+'/feed?access_token='+token #first request
    d = {'ID': [], 'URL': [], 'Publish Date': [], 'Lead sentences': []}
    while (True):
        posts_data = requests.get(request).json()
        for post in posts_data['data']:
            post_date = datetime.strptime(post['created_time'][:10], '%Y-%m-%d')
            if post_date < stop_date:
                break
            d['ID'].append(post['id'])
            d['URL'].append('https://www.facebook.com/'+post['id'])
            d['Publish Date'].append(post_date)
            try:
                lead = post['message'].replace('\r', '').replace('\n', '').replace('|', '')
                d['Lead sentences'].append(lead)
            except:
                lead = '0'
                d['Lead sentences'].append(lead)
        request = posts_data['paging']['next']
        if post_date < stop_date:
            break
    return d

In [3]:
def get_comments_from_posts(post_ids, token):
    comment_message = []
    d = {'ID': [], 'comment': []}
    for post_id in post_ids:
        try:
            comments_on_post = requests.get('https://graph.facebook.com/'+post_id+'/comments?access_token='+token)
            comments_data = comments_on_post.json()
            for c in comments_data['data']:
                d['ID'].append(post_id)
                d['comment'].append(c['message'].replace('\n', '').replace('|', ''))
        except:
            pass
    return d

In [4]:
def fbscrapper(page_id, token, fname, stop_date):
    # скраплю краткую информацию о постах, пишу в csv:
    posts = get_posts_from_page(page_id, token, stop_date)
    print("Собрана информация о постах до даты: ", stop_date, {key: len(value) for key, value in posts.items()})
    df = pd.DataFrame(data = posts, )
    df.to_csv('./csv/fb_' + fname + '_' + str(stop_date) +'_posts.csv', sep='|', index=False, encoding='utf-8')
    # скраплю комментарии, пишу в csv:
    comments = get_comments_from_posts(posts['ID'], token)
    print('Собрано комментариев к постам до даты:', stop_date, {key: len(value) for key, value in comments.items()})
    df = pd.DataFrame(data = comments)
    df.to_csv('./csv/fb_' + fname + '_' + str(stop_date) +'_comments.csv', sep='|', index=False, encoding='utf-8')    
    return None

In [6]:
%%time
token = 'EAACEdEose0cBAIGvZBQd7MtzrOyPPRuFC81l6I95ZA6uR7UROuFa6tbeZBdDwDQ01oBhEBDZAgaaFe3VlE2GJBipwttNF9TKq37DIL9PIy5IbzwSFhZA15kt23qoXHbQEd4DA3yWMYfjcgFIOdo5ZBnCZBQpVkKeC1lrMZCHm4uUvvUzWSvugiTLGnaSXmOO0BPXlqdctKFsfwZDZD'
page_id = '324852143799' #tvrain
fbscrapper(page_id, token, '03.01.2017')

Собрана информация о постах до даты:  01.01.2017 {'ID': 1519, 'URL': 1519, 'Publish Date': 1519, 'Lead sentences': 1519}
Собрано комментариев к постам до даты: 01.01.2017 {'ID': 24507, 'comment': 24507}
Wall time: 10min 10s


In [8]:
df = pd.read_csv('./csv/tvrain_01.01.2017_posts.csv', sep='|', encoding='utf-8')
df.tail()

,ID,Lead sentences,Publish Date,URL
1516,324852143799_10154635722058800,Почему внешняя политика для Путина важнее внут...,2017-02-19,https://www.facebook.com/324852143799_10154635...
1517,324852143799_10154635401233800,"«Именно из-за того, что такой запрет, я это де...",2017-02-19,https://www.facebook.com/324852143799_10154635...
1518,324852143799_10154633543753800,Перебежчики Вороненков и Максакова и реакция р...,2017-02-19,https://www.facebook.com/324852143799_10154633...
1519,324852143799_10154634984613800,К скандинавским детективам все давно уже привы...,2017-02-19,https://www.facebook.com/324852143799_10154634...
1520,324852143799_10154635222633800,"«Культуры этой еще не было тогда, во-вторых, д...",2017-02-19,https://www.facebook.com/324852143799_10154635...


In [5]:
%%time
token = 'EAACEdEose0cBAFm6mET9fwmDZBYuZAUQlXZBaMbZA2DpOteY0f3kLC8hBroEE7MV47oFfc3KoMPU1egLdvSWr85xjevo2Fw5BnDGT29GyKF3rZC7g6U0t2cWs1tpYh9MtGnezo9Y0vtc0ED7ZBCZA3h8wiyZBVsbUWZApG2H2ZB0cbsIuTStzZA5Nmv8Os7ZB9JsiZBdB6T7Oy3lbIAZDZD'
page_id = '98606252058' #snob.ru
fbscrapper(page_id, token, '02.01.2017')

Собрана информация о постах до даты:  02.01.2017 {'ID': 1533, 'URL': 1533, 'Publish Date': 1533, 'Lead sentences': 1533}
Собрано комментариев к постам до даты: 02.01.2017 {'ID': 15150, 'comment': 15150}
Wall time: 11min 59s


In [7]:
%%time
token = 'EAACEdEose0cBAFJYNGu1sI1kJkT0phr7SCuIKo6QhZBGOolM1bzlu5Wgl6lnsciZBPW9ZC9PyOFyoQkRVklP1zwhLpz5aoNeAvQIbSI6ysIr4zgXiujZCQW2iCbeoWZAm4iUz4TSBd8mlgnHqsUgjot3cFGEMuSF0vNRqavEArMJ7e0hWarbbYr6N19yUZA9mIeHBgaZAkfUgZDZD'
page_id = '190542150967819' #life news
fbscrapper(page_id, token, '01.01.2017')


Собрана информация о постах до даты:  01.01.2017 {'ID': 1520, 'URL': 1520, 'Publish Date': 1520, 'Lead sentences': 1520}
Собрано комментариев к постам до даты: 01.01.2017 {'ID': 10039, 'comment': 10039}
Wall time: 9min 52s


In [8]:
%%time
token = 'EAACEdEose0cBAFJYNGu1sI1kJkT0phr7SCuIKo6QhZBGOolM1bzlu5Wgl6lnsciZBPW9ZC9PyOFyoQkRVklP1zwhLpz5aoNeAvQIbSI6ysIr4zgXiujZCQW2iCbeoWZAm4iUz4TSBd8mlgnHqsUgjot3cFGEMuSF0vNRqavEArMJ7e0hWarbbYr6N19yUZA9mIeHBgaZAkfUgZDZD'
page_id = '190542150967819' #life news
fbscrapper(page_id, token, '01.01.2016')

Собрана информация о постах до даты:  01.01.2016 {'ID': 2102, 'URL': 2102, 'Publish Date': 2102, 'Lead sentences': 2102}
Собрано комментариев к постам до даты: 01.01.2016 {'ID': 14014, 'comment': 14014}
Wall time: 13min


In [10]:
%%time
token = 'EAACEdEose0cBANAwBwZAZB0EQhZAeyuXg1SP6U447M0Xqrc58iTZC9xIdT6UIldIwHSWfZAQZCgKGzYNpaD2oFLy7hCmZC4tpyjF2LXR4IQSzyYPdkNqcT1XC0loGp6iRJYZA2bh04bDyalHNKw4vR9OZAN5BhkLr68oYZCnzkBBehxtcdXBMRjYZB4aUkLjDkE3IPyx76qPQuWDgZDZD'
page_id = '325794294270' #ria news
fbscrapper(page_id, token, '02.01.2016')

Собрана информация о постах до даты:  02.01.2016 {'ID': 2140, 'URL': 2140, 'Publish Date': 2140, 'Lead sentences': 2140}
Собрано комментариев к постам до даты: 02.01.2016 {'ID': 36676, 'comment': 36676}
Wall time: 18min 18s


In [12]:
%%time
token = 'EAACEdEose0cBAAsOfsn7M8cPJAEDHzZBCwBZCl9LFJdacW67It8yZAeX7mo62bmwQbFSy1W5eHvZAQwZCM8E2bBa6vHXzhiK4zaebSgYqmlAjze3Q5lPJaBvPYoer3hZBXEjFG43zUVHEkKlsNtgJaJreMuDlfYlhvi8AK5iZCvPyLbMMZBAor7AIlGZBPnMeUmyxKDM1NfJNnwZDZD'
page_id = '375587425929352' #meduza
fbscrapper(page_id, token, '03.01.2016')

Собрана информация о постах до даты:  03.01.2016 {'ID': 2139, 'URL': 2139, 'Publish Date': 2139, 'Lead sentences': 2139}
Собрано комментариев к постам до даты: 03.01.2016 {'ID': 27107, 'comment': 27107}
Wall time: 16min 23s


In [8]:
%%time
token = 'EAACEdEose0cBABl9TwNBcjr02525dgQ8ZCOC3uiQEDrzNMooHENI9WCCowDnDxnwnXmwnj2D1E218SysoHuBZBfz1qld55oRddZB44j95zjy5TAz8zr6EMEytTwzZBY4hdMwgB2ZCsZAaerPCbwFTVV9qD7uZBhNVd8ZAiR9aHxxphoHEz6UlnrF6Dg4ZByg83KFSZAeHJxJEiiwZDZD'
page_id = '127803987254114' #vesti.ru
fbscrapper(page_id, token, 'vesti.ru', '03.01.2015')

Собрана информация о постах до даты:  03.01.2015 {'ID': 2695, 'URL': 2695, 'Publish Date': 2695, 'Lead sentences': 2695}
Собрано комментариев к постам до даты: 03.01.2015 {'ID': 37316, 'comment': 37316}
Wall time: 17min 32s


In [9]:
%%time
token = 'EAACEdEose0cBABl9TwNBcjr02525dgQ8ZCOC3uiQEDrzNMooHENI9WCCowDnDxnwnXmwnj2D1E218SysoHuBZBfz1qld55oRddZB44j95zjy5TAz8zr6EMEytTwzZBY4hdMwgB2ZCsZAaerPCbwFTVV9qD7uZBhNVd8ZAiR9aHxxphoHEz6UlnrF6Dg4ZByg83KFSZAeHJxJEiiwZDZD'
page_id = '184860593907' #vedomosti
fbscrapper(page_id, token, 'vedomosti', '03.01.2015')

Собрана информация о постах до даты:  03.01.2015 {'ID': 2707, 'URL': 2707, 'Publish Date': 2707, 'Lead sentences': 2707}
Собрано комментариев к постам до даты: 03.01.2015 {'ID': 31055, 'comment': 31055}
Wall time: 12min 28s


In [5]:
%%time
token = 'EAACEdEose0cBACa2tnk6tqY1mac53TUoo3cMD8rxPyZBXVZAbZBxLxgmAcEaJsGkZBWs3OyTrLKRlVtUPBMZBg0TMntYPwJTouYjM4jnXtSwro6qZCl099RlaNNo0QG3XAoUq7LN9KlfX2ItXZCePhxtJu3IYgKmzz0u7P6rXT2MJsg1n2mcaTVTlGmnlJZAAZC7UIwNAcLGyrAZDZD'
page_id = '182549563410' #lentaru
fbscrapper(page_id, token, 'lentaru', '03.01.2015')

Собрана информация о постах до даты:  03.01.2015 {'ID': 2732, 'URL': 2732, 'Publish Date': 2732, 'Lead sentences': 2732}
Собрано комментариев к постам до даты: 03.01.2015 {'ID': 22969, 'comment': 22969}
Wall time: 15min 6s


In [6]:
%%time
token = 'EAACEdEose0cBACa2tnk6tqY1mac53TUoo3cMD8rxPyZBXVZAbZBxLxgmAcEaJsGkZBWs3OyTrLKRlVtUPBMZBg0TMntYPwJTouYjM4jnXtSwro6qZCl099RlaNNo0QG3XAoUq7LN9KlfX2ItXZCePhxtJu3IYgKmzz0u7P6rXT2MJsg1n2mcaTVTlGmnlJZAAZC7UIwNAcLGyrAZDZD'
page_id = '286029632119' #gazeta.ru
fbscrapper(page_id, token, 'gazeta.ru', '03.01.2015')

Собрана информация о постах до даты:  03.01.2015 {'ID': 2756, 'URL': 2756, 'Publish Date': 2756, 'Lead sentences': 2756}
Собрано комментариев к постам до даты: 03.01.2015 {'ID': 15849, 'comment': 15849}
Wall time: 12min 34s


In [20]:
%%time
token = 'EAACEdEose0cBAAq3R2ZARzjIZC8NzjCyyBHMfoStC1veuepPMPirCdZCLPZA3ygGU43BZAAGYVbnJa01LGep3HZAHWDnl3oBFCJiAUmnujk7M2wMEI6AS4GtVc6ofiyZCOo85FmALvqqZBYeWhRVE5trjPZAl1TSJ1ZBBUwo7zOCSC4miHv7hqwXr6kuR43l1QEErPSbtjnjf8HwZDZD'
page_id = '112954062056852' #1tvru
fbscrapper(page_id, token, '1tvru', '03.01.2015')

Собрана информация о постах до даты:  03.01.2015 {'ID': 2547, 'URL': 2547, 'Publish Date': 2547, 'Lead sentences': 2547}
Собрано комментариев к постам до даты: 03.01.2015 {'ID': 33473, 'comment': 33473}
Wall time: 10min 43s


In [8]:
%%time
token = 'EAACEdEose0cBAFeuvarIEH8TIZBVwptUdW8ZAqjqIgJc8KTeVZAxc7QWrlupKy7KmPtRou71rf1W0xYfDdOIzdhQZAxtIYKyXHgJ9AMZCKZCTsMBvfkh5siKOnQv8TfAnICNOcoyZB6lrlZBaMoCkd8nbV8xAi0omZC7KPxtqNswav3fdYmUI4OBILP0GFAOBLUjWk0yjlMOkoAZDZD'
page_id = '135161613191462' #aif.ru
fbscrapper(page_id, token, 'aif.ru', '03.01.2015')

Собрана информация о постах до даты:  03.01.2015 {'ID': 2753, 'URL': 2753, 'Publish Date': 2753, 'Lead sentences': 2753}
Собрано комментариев к постам до даты: 03.01.2015 {'ID': 20075, 'comment': 20075}
Wall time: 12min 34s


In [9]:
%%time
token = 'EAACEdEose0cBAFeuvarIEH8TIZBVwptUdW8ZAqjqIgJc8KTeVZAxc7QWrlupKy7KmPtRou71rf1W0xYfDdOIzdhQZAxtIYKyXHgJ9AMZCKZCTsMBvfkh5siKOnQv8TfAnICNOcoyZB6lrlZBaMoCkd8nbV8xAi0omZC7KPxtqNswav3fdYmUI4OBILP0GFAOBLUjWk0yjlMOkoAZDZD'
page_id = '369256943469' #onlinekpru
fbscrapper(page_id, token, 'onlinekpru', '03.01.2015')

Собрана информация о постах до даты:  03.01.2015 {'ID': 2672, 'URL': 2672, 'Publish Date': 2672, 'Lead sentences': 2672}
Собрано комментариев к постам до даты: 03.01.2015 {'ID': 23378, 'comment': 23378}
Wall time: 11min 33s


In [10]:
%%time
token = 'EAACEdEose0cBAFeuvarIEH8TIZBVwptUdW8ZAqjqIgJc8KTeVZAxc7QWrlupKy7KmPtRou71rf1W0xYfDdOIzdhQZAxtIYKyXHgJ9AMZCKZCTsMBvfkh5siKOnQv8TfAnICNOcoyZB6lrlZBaMoCkd8nbV8xAi0omZC7KPxtqNswav3fdYmUI4OBILP0GFAOBLUjWk0yjlMOkoAZDZD'
page_id = '140965502635514' #rbc.ru
fbscrapper(page_id, token, 'rbc.ru', '03.01.2015')

Собрана информация о постах до даты:  03.01.2015 {'ID': 2725, 'URL': 2725, 'Publish Date': 2725, 'Lead sentences': 2725}
Собрано комментариев к постам до даты: 03.01.2015 {'ID': 45440, 'comment': 45440}
Wall time: 12min 48s


In [11]:
%%time
token = 'EAACEdEose0cBAAprodzRKa6ZB0hQxIKsdov43ITSxTiQfBczyZAZCXwmBY0kBHmgOx2XwJiuFCpyjydFurih9vKyjc8avW4HAb3Q6ZBq84AO1uiaDhqmaT3UDbrP6CaUz5ROmEHVa2R1HZA0O7lvrWZAE4jn0ZAAocDvMeC461Aqi3h9lH6r9LUdXTK6BJM6ZCmKNj01O85fZBwZDZD'
page_id = '107044359321519' #InoSMI
fbscrapper(page_id, token, 'InoSMI', '03.01.2015')

Собрана информация о постах до даты:  03.01.2015 {'ID': 2431, 'URL': 2431, 'Publish Date': 2431, 'Lead sentences': 2431}
Собрано комментариев к постам до даты: 03.01.2015 {'ID': 30334, 'comment': 30334}
Wall time: 11min 8s


In [12]:
%%time
token = 'EAACEdEose0cBAAprodzRKa6ZB0hQxIKsdov43ITSxTiQfBczyZAZCXwmBY0kBHmgOx2XwJiuFCpyjydFurih9vKyjc8avW4HAb3Q6ZBq84AO1uiaDhqmaT3UDbrP6CaUz5ROmEHVa2R1HZA0O7lvrWZAE4jn0ZAAocDvMeC461Aqi3h9lH6r9LUdXTK6BJM6ZCmKNj01O85fZBwZDZD'
page_id = '139349786856' #kommersant.ru
fbscrapper(page_id, token, 'kommersant.ru', '03.01.2015')

Собрана информация о постах до даты:  03.01.2015 {'ID': 2744, 'URL': 2744, 'Publish Date': 2744, 'Lead sentences': 2744}
Собрано комментариев к постам до даты: 03.01.2015 {'ID': 9794, 'comment': 9794}
Wall time: 11min 5s
